In [19]:
#导入数据
test = []
allWords_test = []
with open('text.txt', 'r', encoding='utf-8') as f:
    for i in f.readlines():
        #得到每一行的文本数据并按照空格切分为单词
        text = i.split('\t')[2].split(' ')
        #去除最后一个单词的换行符
        text[-1] = text[-1].strip('\n')  
        test.append(text) 
        for word in text:
            #若该词未出现过，则将其添加到allWords中
            if word not in allWords_test: 
                #将词转换成小写来进行判断，以防出现A和a不分的情况
                allWords_test.append(word) 
test
allWords_test

[['say', 'you', 'say', 'me'],
 ['lay', 'me', 'down'],
 ['stay', 'with', 'me'],
 ['because', 'you', 'loved', 'me'],
 ['me', 'I', 'my', 'mine', 'me', 'my', 'me'],
 ['you', 'can', 'trust', 'me', 'can', 'you']]

['say',
 'you',
 'me',
 'lay',
 'down',
 'stay',
 'with',
 'because',
 'loved',
 'I',
 'my',
 'mine',
 'can',
 'trust']

In [17]:
len(allWords_test)

14

In [20]:
def getOneHot(dataSet, allWords):
    '''得到输入数据集的one-hot矩阵'''
    oneHot=[]
    for index, doc in enumerate(dataSet):
        oneHot.append([])
        for word in allWords:
            if word in doc:
                oneHot[index].append(1)
            else: 
                oneHot[index].append(0)
    return oneHot

from collections import Counter
from math import log
import numpy as np

def getTF(dataSet, allWords):
    '''得到输入数据集的TF矩阵'''
    TF=[]
    for index, doc in enumerate(dataSet):
        TF.append([])
        wordCounter = Counter(doc)
        for word in allWords:
            TF[index].append(wordCounter.get(word,0)/len(doc))
    return TF


def getIDF(dataSet, allWords):
    '''得到输入数据集的IDF矩阵'''
    def calcIDF(num):
        '''计算对应数据集的单词的IDF值'''
        return log(len(dataSet)/(1+num))
    
    IDF=[]
    for word in allWords:
        cnt = 0
        #计算词在每个文档出现的次数
        for doc in dataSet: 
            if word in doc:
                cnt += 1
        IDF.append(calcIDF(cnt))
    return IDF

def saveDataToFile(data, fileName):
    '''保存数据到文件中'''
    with open(fileName, 'w', encoding='utf-8') as f:
        for i in data:
            #直接将list类型的元素保存到文件比双重循环写文件更快
            _ = f.write(str(i)+'\n')
            

def getTFIDF(TF, IDF):
    '''得到输入数据集的TF-IDF矩阵'''
    TF=np.array(TF)   #先将数据类型转换为numpy的array类型
    IDF=np.array(IDF) #以便实现numpy广播运算
    return (TF*IDF).tolist() #转换为list类型

def sparseMatrix2TripleTable(sMatrix):
    '''将输入的稀疏矩阵转换为三元组'''
    sMatrix = np.array(sMatrix) #转换为numpy的数组类型
    tripleTable = [[0],[0],[0]]
    tripleTable[0][0], tripleTable[1][0] = sMatrix.shape #得到行数、列数
   
    nonZeroIndex = np.nonzero(sMatrix) #得到矩阵非零元素的下标组成的tuple
    
    nonZeroNum = len(nonZeroIndex[0]) #记录非零个数
    tripleTable[2][0] = nonZeroNum 
    
    for i in range(nonZeroNum):
        rowIndex = nonZeroIndex[0][i]
        colIndex = nonZeroIndex[1][i]
        val = sMatrix[rowIndex, colIndex]
        tripleTable.append([rowIndex, colIndex, val])
    return tripleTable

def addSparseMatrix(triTable1, triTable2):
    '''两个行和列数相等的稀疏矩阵（以三元顺序组表示）的加法'''
    ans = triTable1
    vistedList = [] #标记访问过的三元组的下标
    
    for i, val1 in enumerate(triTable1[3:]):
        for j, val2 in enumerate(triTable2[3:]):
            #若 j还没有被访问 且 两个矩阵在相同的位置上都有非零元素
            if j not in vistedList and (val1[0], val1[1]) == (val2[0], val2[1]):
                ans[i+3][-1] += val2[-1]
                vistedList.append(j) #标记已访问
    #得到未访问的三元组下标
    unVistedList = [i for i in range(triTable2[2][0]) if i not in vistedList]
    ans[2][0] += len(unVistedList)#更新结果的非零元素个数
    #将第二个
    for i in unVistedList:
        ans.append(triTable2[i+3])
    #将部分结果排好序后再返回
    temp = ans[3:]
    temp.sort(key = lambda x: (x[0],x[1]))
    return ans[:3]+temp

oneHot_test = getOneHot(test, allWords_test)
saveDataToFile(oneHot_test, 'onehot_test.txt')

TF_test = getTF(test, allWords_test)
saveDataToFile(TF_test, 'tf_test.txt')

IDF_test = getIDF(test, allWords_test)
TFIDF_test = getTFIDF(TF_test, IDF_test)
saveDataToFile(TFIDF_test, 'tfidf_test.txt')

tripleTable_test = sparseMatrix2TripleTable(oneHot_test)
saveDataToFile(tripleTable_test, 'smatrix_test.txt')

In [21]:
import numpy as np

def addSparseMatrix(triTable1, triTable2):
    '''两个行和列数相等的稀疏矩阵（以三元顺序组表示）的加法'''
    ans = triTable1
    vistedList = [] #标记访问过的三元组的下标
    
    for i, val1 in enumerate(triTable1[3:]):
        for j, val2 in enumerate(triTable2[3:]):
            #若 j还没有被访问 且 两个矩阵在相同的位置上都有非零元素
            if j not in vistedList and (val1[0], val1[1]) == (val2[0], val2[1]):
                ans[i+3][-1] += val2[-1]
                vistedList.append(j) #标记已访问
    #得到未访问的三元组下标
    unVistedList = [i for i in range(triTable2[2][0]) if i not in vistedList]
    ans[2][0] += len(unVistedList)#更新结果的非零元素个数
    #将第二个
    for i in unVistedList:
        ans.append(triTable2[i+3])
    #将部分结果排好序后再返回
    temp = ans[3:]
    temp.sort(key = lambda x: (x[0],x[1]))
    return ans[:3]+temp

def readTestFile(fileName):
    A=[]
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            A.append(eval('['+line.replace(' ',',')+']'))
    return A

tripleTable_test1 = readTestFile('A.txt')
tripleTable_test2 = readTestFile('B.txt')

addRes = addSparseMatrix(tripleTable_test1, tripleTable_test2)
addRes
saveDataToFile(addRes, 'A+B.txt')

[[3],
 [14],
 [17],
 [0, 0, 2],
 [0, 1, 2],
 [0, 2, 2],
 [0, 7, 1],
 [0, 8, 1],
 [1, 2, 4],
 [1, 3, 1],
 [1, 4, 1],
 [1, 9, 1],
 [1, 10, 2],
 [1, 11, 1],
 [2, 1, 2],
 [2, 2, 2],
 [2, 5, 1],
 [2, 6, 1],
 [2, 12, 2],
 [2, 13, 1]]

In [18]:
IDF_test
allWords_test

[1.0986122886681098,
 0.4054651081081644,
 -0.15415067982725836,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098,
 1.791759469228055,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098]

['say',
 'you',
 'me',
 'lay',
 'down',
 'stay',
 'with',
 'because',
 'loved',
 'i',
 'my',
 'mine',
 'can',
 'trust']

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"